In [1]:
import random

class Sentiment:
    POSITIVE="POSITIVE"
    NEUTRAL="NEUTRAL"
    NEGATIVE="NEGATIVE"


class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiment()
        
    def get_sentiment(self):
        if self.score>=4:
            return Sentiment.POSITIVE
        elif self.score==3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.NEGATIVE
        
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative=list(filter(lambda x:x.sentiment==Sentiment.NEGATIVE,self.reviews))
        positive=list(filter(lambda x:x.sentiment==Sentiment.POSITIVE,self.reviews))
        positive_shrunk=positive[:len(negative)]
        self.reviews=negative+positive_shrunk
        random.shuffle(self.reviews)
        

In [2]:
import json

file_name="./Automotive_5.json"

reviews=[]
with open(file_name) as f:
    for line in f:
        review=json.loads(line)
        reviews.append(Review(review['reviewText'],review["overall"]))

reviews[5].sentiment      
        

'POSITIVE'

In [3]:
#preprocessing

from sklearn.model_selection import train_test_split
training,test=train_test_split(reviews,test_size=0.3,random_state=42)

train_cont=ReviewContainer(training)
train_cont.evenly_distribute()
test_cont=ReviewContainer(test)
test_cont.evenly_distribute()

x_train=train_cont.get_text()
y_train=train_cont.get_sentiment()

x_test=test_cont.get_text()
y_test=test_cont.get_sentiment()

print(y_train.count(Sentiment.POSITIVE))
print(y_train.count(Sentiment.NEGATIVE))

800
800


In [4]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

vectorizer=TfidfVectorizer()
x_train_vector=vectorizer.fit_transform(x_train)
x_test_vector=vectorizer.transform(x_test)

x_train[0]


'I\'ve been detailing and cleaning my vehicles for the past 15 years. So I\'m rather particular about what I use and how I clean the vehicles. Which is why I used this on a friends vehicle  (with their permission) who doesn\'t particularly care about how it looks. :)First of all, you should NEVER use any product like this on a dirty vehicle without washing it first. Otherwise you are simply rubbing dirt directly into the paint and you will have swirl marks. Especially if you have a dark colored vehicle. The vehicle should idealy be washed with pressure washer to remove large/loose dirt (or reputable touchless wash if not available), and then with a wash mit and bucket (with grit guard) using car wash soap. If it\'s in the kitchen, don\'t use it. Car wash soap only please.Once you have a 99% clean vehicle, you may then proceed to use the Lucky Earth detailing kit. You can use this to clean up any spots you may have missed on the paint, wheels, or even glass if you feel like it. For this

In [5]:
#linear svm


from sklearn import svm

clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(x_train_vector,y_train)

clf_svm.predict(x_test_vector[0])


array(['POSITIVE'], dtype='<U8')

In [6]:
print(clf_svm.score(x_test_vector,y_test))

0.7859195402298851


In [7]:
test_set=['this is very bad','good']
new_test=vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['NEGATIVE', 'POSITIVE'], dtype='<U8')

In [8]:
import pickle

In [9]:
pickle.dump(clf_svm,open('sa.pkl','wb'))

In [10]:
pickle.dump(vectorizer,open('vect.pkl','wb'))

In [11]:
r=pickle.load(open('vect.pkl','rb'))

In [12]:
clf_svm.predict(r.transform(['it is bad']))

array(['NEGATIVE'], dtype='<U8')